In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn as nn
import torch

In [2]:
path='/kaggle/input/digit-recognizer/train.csv'
trainset=pd.read_csv(path)
trainset

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from torch.utils.data import DataLoader

In [4]:
trainset.drop('label',axis=1)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#DataLoader.dataset()

## 1-Data

In [6]:
train_tensor=torch.from_numpy(np.array(trainset))

In [7]:
train_tensor

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [7, 0, 0,  ..., 0, 0, 0],
        [6, 0, 0,  ..., 0, 0, 0],
        [9, 0, 0,  ..., 0, 0, 0]])

In [8]:
from torch.utils.data import Dataset

In [9]:
class digits_dataset(Dataset):
    def __init__(self):
        data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
        self.y=torch.from_numpy(np.array(data['label']))
        self.x=torch.from_numpy(np.array(data.drop('label',axis=1))/255)
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [10]:
data=digits_dataset()

In [11]:
train ,test = torch.utils.data.random_split(data,[0.8,0.2])

In [12]:
x,y=train[0]

In [13]:
train.__len__()

33600

In [14]:
print(x.shape)

torch.Size([784])


## 2-Neural Network and Model

In [15]:
train_loader=DataLoader(dataset=train,batch_size=200,shuffle=True)

In [16]:
model=nn.Sequential(nn.Linear(28*28,512),
                    nn.ReLU(),
                    nn.Linear(512,512),
                    nn.ReLU(),
                    nn.Linear(512,10)
                    )

## 3-Training 

In [17]:
criterion=nn.CrossEntropyLoss()

In [18]:
import torch.optim as optim

In [19]:
optimizer=optim.Adam(params=model.parameters(),lr=0.001)

### a-Train loop 

In [20]:
for epoch in range(10):
    training_loss = 0
    for data in train_loader:
        #setting gradient at zero
        optimizer.zero_grad()
        #getting data
        features,label=data
        features=features.float()
        #predicting class
        pred=model(features)
        #loss
        loss=criterion(pred,label)
        loss.backward()
        #updating params
        optimizer.step()
        training_loss += loss.item()
    epoch_loss = training_loss / len(train_loader)
    print(f'loss {epoch_loss} for {epoch+1}/10')


loss 0.41771468342769713 for 1/10
loss 0.15333103224457728 for 2/10
loss 0.09554369550287015 for 3/10
loss 0.06444449596927457 for 4/10
loss 0.044438212599960114 for 5/10
loss 0.033202021795746293 for 6/10
loss 0.023626616912051326 for 7/10
loss 0.01704664444938923 for 8/10
loss 0.012239720851842625 for 9/10
loss 0.007142530762523945 for 10/10


### b-Validation loop

In [21]:
test_loader=DataLoader(dataset=test,batch_size=200,shuffle=False)

In [22]:
validation_loss = 0.0
model.eval() # Put model in evaluation mode
with torch.no_grad(): # Speed up the forward pass
    training_loss = 0
    for data in test_loader:
        #getting data
        features,label=data
        features=features.float()
        #predicting class
        pred=model(features)
        #loss
        loss=criterion(pred,label)
        validation_loss += loss.item()
    validation_loss = validation_loss / len(test_loader)
    print(f'validation loss is {validation_loss}')
model.train()


validation loss is 0.10282895731784049


Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=10, bias=True)
)

In [23]:
import torchmetrics
# Create accuracy metric using torch metrics
metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)
for i, data in enumerate(test_loader, 0):
    features, labels = data
    features=features.float()
    outputs = model(features)
    # Calculate accuracy over the batch
    acc = metric(outputs, labels)
# Calculate accuracy over the whole epoch
acc = metric.compute()
print(f"Accuracy on all data: {acc}")
# Reset the metric for the next epoch (training or validation)
metric.reset()

Accuracy on all data: 0.9776190519332886
